In [1]:
print('hello')

hello


## Best practices
1. Preprocessing And Cleaning 
2. Train Test Split 
3. Bow,TFIDF,Word2vec
4. Train ML algorithms

In [92]:
import pandas as pd 
df=pd.read_csv('dataset/kindle_reviews.csv')


In [93]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [94]:
df=df[['reviewText','overall']]

In [95]:
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [96]:
df.shape

(982619, 2)

In [97]:
##check missing values
df.isnull().sum()

reviewText    22
overall        0
dtype: int64

In [98]:
df['overall'].unique()

array([5, 4, 3, 2, 1], dtype=int64)

In [99]:
#check index for null values
null_indexes = df[df.isnull().any(axis=1)].index
print(null_indexes.tolist())


[32652, 143683, 163443, 193140, 198536, 255972, 308493, 312057, 312310, 370839, 407344, 407624, 509549, 513873, 548273, 587053, 679623, 719610, 719613, 719620, 725868, 748161]


In [100]:
print(df.loc[null_indexes.tolist()])

       reviewText  overall
32652         NaN        5
143683        NaN        5
163443        NaN        5
193140        NaN        3
198536        NaN        5
255972        NaN        5
308493        NaN        4
312057        NaN        5
312310        NaN        5
370839        NaN        4
407344        NaN        5
407624        NaN        5
509549        NaN        5
513873        NaN        5
548273        NaN        5
587053        NaN        4
679623        NaN        5
719610        NaN        5
719613        NaN        5
719620        NaN        5
725868        NaN        5
748161        NaN        5


In [101]:
# Replace NaN in reviewText based on overall rating
df.loc[df['reviewText'].isna() & (df['overall'] == 5), 'reviewText'] = 'super'
df.loc[df['reviewText'].isna() & (df['overall'] == 4), 'reviewText'] = 'good'
df.loc[df['reviewText'].isna() & (df['overall'] == 3), 'reviewText'] = 'average'


In [102]:
print(df.loc[null_indexes.tolist()])

       reviewText  overall
32652       super        5
143683      super        5
163443      super        5
193140    average        3
198536      super        5
255972      super        5
308493       good        4
312057      super        5
312310      super        5
370839       good        4
407344      super        5
407624      super        5
509549      super        5
513873      super        5
548273      super        5
587053       good        4
679623      super        5
719610      super        5
719613      super        5
719620      super        5
725868      super        5
748161      super        5


In [103]:
#now check
df.isnull().sum()

reviewText    0
overall       0
dtype: int64

In [104]:
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [105]:
df['overall'].value_counts()

overall
5    575264
4    254013
3     96194
2     34130
1     23018
Name: count, dtype: int64

# PREPROCESSING AND CLEANING

In [106]:
#if overall rating is greater than 3 -- positive rating and if it less than 3 -- negative rating

In [107]:
## positive review is 1 and negative review is 0 
df['overall']=df['overall'].apply(lambda x:0 if x<3 else 1)
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,1
1,This book is a reissue of an old one; the auth...,1
2,This was a fairly interesting read. It had ol...,1
3,I'd never read any of the Amy Brewster mysteri...,1
4,"If you like period pieces - clothing, lingo, y...",1


In [108]:
df['overall'].value_counts()

overall
1    925471
0     57148
Name: count, dtype: int64

In [109]:
## 1. Lower all the cases
df['reviewText']=df['reviewText'].str.lower()

In [110]:
df.head()

,reviewText,overall
0,i enjoy vintage books and movies so i enjoyed ...,1
1,this book is a reissue of an old one; the auth...,1
2,this was a fairly interesting read. it had ol...,1
3,i'd never read any of the amy brewster mysteri...,1
4,"if you like period pieces - clothing, lingo, y...",1


In [111]:
#2
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
# Ensure stopwords are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [112]:
# Remove special characters (keep spaces)
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]+', ' ', str(x)))


In [113]:
# Remove URLs
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', str(x)))

In [114]:
# Remove HTML tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())

In [115]:
# Remove extra spaces (convert multiple spaces to single space)
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())


In [116]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y.lower() not in stop_words]))

In [117]:
df.head()

,reviewText,overall
0,enjoy vintage books movies enjoyed reading boo...,1
1,book reissue old one author born 1910 era say ...,1
2,fairly interesting read old style terminology ...,1
3,never read amy brewster mysteries one really h...,1
4,like period pieces clothing lingo enjoy myster...,1


In [118]:
## lemmatizer
from nltk.stem import WordNetLemmatizer

In [119]:
lemmatizer=WordNetLemmatizer()

In [120]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))


In [121]:
df.head()

,reviewText,overall
0,enjoy vintage book movie enjoyed reading book ...,1
1,book reissue old one author born 1910 era say ...,1
2,fairly interesting read old style terminology ...,1
3,never read amy brewster mystery one really hooked,1
4,like period piece clothing lingo enjoy mystery...,1


In [122]:
df.to_csv('df_up.csv',index=False) ###########saving the dataset containing vocabulary

In [123]:
## Train Test split 
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(df['reviewText'],df['overall'],test_size=0.25)

In [124]:
##################### BOW
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
bow=CountVectorizer()
x_train_bow=bow.fit_transform(x_train)
x_test_bow=bow.transform(x_test)

In [125]:
##################### tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf=tfidf.transform(x_test)

In [126]:
x_train_bow

<736964x228469 sparse matrix of type '<class 'numpy.int64'>'
	with 32437335 stored elements in Compressed Sparse Row format>

In [127]:
###now use ml model MultinomialNB
from sklearn.naive_bayes import MultinomialNB 
mnb_model_bow=MultinomialNB().fit(x_train_bow,y_train)
mnb_model_tfidf=MultinomialNB().fit(x_train_tfidf,y_train)
###performance
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
#prediction bow
y_pred_bow_test=mnb_model_bow.predict(x_test_bow)
y_pred_bow_train=mnb_model_bow.predict(x_train_bow)
#prediction tfidf
y_pred_tfidf_test=mnb_model_tfidf.predict(x_test_tfidf)
y_pred_tfidf_train=mnb_model_tfidf.predict(x_train_tfidf)
print('-------------------BOW------------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_bow_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_bow_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_bow_test)}')
print(f'classification report:{classification_report(y_test,y_pred_bow_test)}')

print('-------------------tfidf----------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_tfidf_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_tfidf_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_tfidf_test)}')
print(f'classification report :{classification_report(y_test,y_pred_tfidf_test)}')


import pickle
with open("mnb_model_bow.pkl", "wb") as f:
    pickle.dump(mnb_model_bow, f)
    
with open("mnb_model_tfidf.pkl", "wb") as f:
    pickle.dump(mnb_model_tfidf, f)


-------------------BOW------------------------------------------------
Training accuracy:0.9415968758311125
Testing accuracy:0.941560318332621
confusion matrix:[[  8295   5858]
 [  8498 223004]]
classification report:              precision    recall  f1-score   support

           0       0.49      0.59      0.54     14153
           1       0.97      0.96      0.97    231502

    accuracy                           0.94    245655
   macro avg       0.73      0.77      0.75    245655
weighted avg       0.95      0.94      0.94    245655

-------------------tfidf----------------------------------------------
Training accuracy:0.9414706824213938
Testing accuracy:0.9422238505220737
confusion matrix:[[    44  14109]
 [    84 231418]]
classification report :              precision    recall  f1-score   support

           0       0.34      0.00      0.01     14153
           1       0.94      1.00      0.97    231502

    accuracy                           0.94    245655
   macro avg       

In [128]:
###now use ml model logistic regression
from sklearn.linear_model import LogisticRegression

lgr_model_bow=LogisticRegression().fit(x_train_bow,y_train)
lgr_model_tfidf=LogisticRegression().fit(x_train_tfidf,y_train)
###performance
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
#prediction bow
y_pred_bow_test=lgr_model_bow.predict(x_test_bow)
y_pred_bow_train=lgr_model_bow.predict(x_train_bow)
#prediction tfidf
y_pred_tfidf_test=lgr_model_tfidf.predict(x_test_tfidf)
y_pred_tfidf_train=lgr_model_tfidf.predict(x_train_tfidf)
print('-------------------BOW------------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_bow_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_bow_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_bow_test)}')
print(f'classification report:{classification_report(y_test,y_pred_bow_test)}')

print('-------------------tfidf----------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_tfidf_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_tfidf_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_tfidf_test)}')
print(f'classification report :{classification_report(y_test,y_pred_tfidf_test)}')


import pickle
with open("lgr_model_bow.pkl", "wb") as f:
    pickle.dump(lgr_model_bow, f)
    
with open("lgr_model_tfidf.pkl", "wb") as f:
    pickle.dump(lgr_model_tfidf, f)



c:\Users\ACER\Desktop\code_with_harry\projects\NLP\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


-------------------BOW------------------------------------------------
Training accuracy:0.9637499253694889
Testing accuracy:0.9574810201298569
confusion matrix:[[  6580   7573]
 [  2872 228630]]
classification report:              precision    recall  f1-score   support

           0       0.70      0.46      0.56     14153
           1       0.97      0.99      0.98    231502

    accuracy                           0.96    245655
   macro avg       0.83      0.73      0.77    245655
weighted avg       0.95      0.96      0.95    245655

-------------------tfidf----------------------------------------------
Training accuracy:0.9605340287992358
Testing accuracy:0.958783660010991
confusion matrix:[[  6261   7892]
 [  2233 229269]]
classification report :              precision    recall  f1-score   support

           0       0.74      0.44      0.55     14153
           1       0.97      0.99      0.98    231502

    accuracy                           0.96    245655
   macro avg       

In [ ]:
###################################################################################  word2vec #############################################################

In [2]:
import pandas as pd 
df=pd.read_csv('df_up.csv')

In [3]:
df.head()

,reviewText,overall
0,enjoy vintage book movie enjoyed reading book ...,1
1,book reissue old one author born 1910 era say ...,1
2,fairly interesting read old style terminology ...,1
3,never read amy brewster mystery one really hooked,1
4,like period piece clothing lingo enjoy mystery...,1


In [4]:
################word2vec##############
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # Added wordnet download for lemmatizer

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load dataset
messages = df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Create cleaned corpus

corpus=[]
for i in range(len(df)):
    corpus.append(messages['reviewText'][i])

In [6]:
corpus

['enjoy vintage book movie enjoyed reading book plot unusual think killing someone self defense leaving scene body without notifying police hitting someone jaw knock would wash today still good read',
 'book reissue old one author born 1910 era say nero wolfe introduction quite interesting explaining author forgotten never heard language little dated time like calling gun 34 heater 34 also made good use fire dictionary look word like 34 deshabille 34 34 canarsie 34 still well worth look see',
 'fairly interesting read old style terminology glad get read story coarse crasslanguage read fun relaxation like free ebooksbecause check writer decide intriguing innovative enough command englishthat convey story without crude language',
 'never read amy brewster mystery one really hooked',
 'like period piece clothing lingo enjoy mystery author guessing least 2 3 way',
 'beautiful depth character description make like fast pacing movie pity mr merwin write 30 instead 3 amy brewster mystery',
 '

In [7]:
import nltk
corpus_words=[]
for sentence in corpus:
    words=nltk.word_tokenize(str(sentence))
    corpus_words.append(words)

In [8]:
corpus_words

[['enjoy',
  'vintage',
  'book',
  'movie',
  'enjoyed',
  'reading',
  'book',
  'plot',
  'unusual',
  'think',
  'killing',
  'someone',
  'self',
  'defense',
  'leaving',
  'scene',
  'body',
  'without',
  'notifying',
  'police',
  'hitting',
  'someone',
  'jaw',
  'knock',
  'would',
  'wash',
  'today',
  'still',
  'good',
  'read'],
 ['book',
  'reissue',
  'old',
  'one',
  'author',
  'born',
  '1910',
  'era',
  'say',
  'nero',
  'wolfe',
  'introduction',
  'quite',
  'interesting',
  'explaining',
  'author',
  'forgotten',
  'never',
  'heard',
  'language',
  'little',
  'dated',
  'time',
  'like',
  'calling',
  'gun',
  '34',
  'heater',
  '34',
  'also',
  'made',
  'good',
  'use',
  'fire',
  'dictionary',
  'look',
  'word',
  'like',
  '34',
  'deshabille',
  '34',
  '34',
  'canarsie',
  '34',
  'still',
  'well',
  'worth',
  'look',
  'see'],
 ['fairly',
  'interesting',
  'read',
  'old',
  'style',
  'terminology',
  'glad',
  'get',
  'read',
  'story

In [9]:
#word2vec model
import gensim
model=gensim.models.Word2Vec(corpus_words)

In [10]:
model.save('word2vec.h5')

In [11]:
### to get all the vocabulary
model.wv.index_to_key #highest frqency at first vocabulary


['book',
 'story',
 'read',
 'one',
 'love',
 'character',
 'like',
 'really',
 'good',
 'series',
 'get',
 'author',
 'great',
 'would',
 'time',
 'well',
 'reading',
 'life',
 'first',
 '8217',
 'way',
 'know',
 'loved',
 'want',
 'make',
 'much',
 'thing',
 'see',
 'even',
 'find',
 'end',
 'could',
 'also',
 'little',
 'next',
 '34',
 'two',
 'enjoyed',
 'go',
 'think',
 'romance',
 'short',
 'lot',
 'going',
 'take',
 'come',
 'new',
 'review',
 'wait',
 'feel',
 'written',
 'say',
 'back',
 'keep',
 'recommend',
 'many',
 'give',
 'part',
 'never',
 'friend',
 'work',
 'need',
 'another',
 'people',
 'thought',
 'put',
 'year',
 'reader',
 'still',
 'made',
 'woman',
 'world',
 'liked',
 'found',
 'man',
 'start',
 'right',
 'hot',
 'relationship',
 'family',
 'something',
 'writing',
 'bit',
 'together',
 'novel',
 'help',
 'looking',
 'got',
 'page',
 'plot',
 'star',
 'though',
 'definitely',
 'best',
 'always',
 'interesting',
 'better',
 'fun',
 'day',
 'sex',
 'heart',
 'di

In [12]:
model.corpus_count #how many sentences

982619

In [13]:
model.epochs

5

In [14]:
model.wv.similar_by_word('good')

[('great', 0.8171629905700684),
 ('decent', 0.7920368909835815),
 ('nice', 0.7109372615814209),
 ('excellent', 0.6258901357650757),
 ('ok', 0.6237711310386658),
 ('alright', 0.606879711151123),
 ('awesome', 0.6054664850234985),
 ('interesting', 0.6021955013275146),
 ('okay', 0.600357711315155),
 ('like', 0.5933901071548462)]

In [15]:
model.wv['free'].shape ##dimensions each word

(100,)

In [16]:
model.wv['enjoy'] #printing the 100 dimensions

array([-0.01898313, -0.8160791 ,  1.6872319 , -1.565915  , -1.112045  ,
       -0.2662329 ,  0.7769203 , -1.2839588 ,  1.3210026 , -0.20943555,
       -1.7155353 ,  3.708781  ,  1.1843506 , -1.6933719 ,  1.3049015 ,
        0.12958506, -1.2858247 , -0.7747076 ,  1.4888915 ,  3.1600916 ,
       -1.3719422 , -1.3424449 , -0.45329854,  2.4575663 ,  2.4294066 ,
       -0.28400236,  3.2358    , -1.0176533 ,  1.6533039 , -2.2503068 ,
       -0.64412194,  0.5373475 , -2.087449  ,  0.5138941 , -1.7303777 ,
       -0.9659017 , -2.8173556 ,  1.0770785 ,  0.26348832,  1.4615662 ,
        0.6342131 ,  2.140176  , -0.46270123,  1.0458567 ,  0.15856431,
        0.12362929,  1.908791  ,  0.73541075, -1.4562865 , -0.5765714 ,
       -0.40226412,  0.8095363 , -1.9315611 ,  2.3044736 , -2.4892883 ,
       -0.4881706 , -0.34695202,  0.85859317, -0.6325882 ,  1.0922418 ,
       -0.42731905, -1.0777593 ,  0.89951986, -1.576904  , -0.16295616,
       -0.96843255,  0.8519462 , -2.7405531 ,  0.46896273,  2.20

In [17]:

import numpy as np
from tqdm import tqdm

def avg_word2vec(doc, model):
    # collect embeddings of words present in the model
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)   # average embedding
    else:
        # if no word found, return a zero vector of same size as embedding
        return np.zeros(model.vector_size)

In [18]:
X = []
for sentence in tqdm(corpus_words):
    X.append(avg_word2vec(sentence, model))

100%|██████████| 982619/982619 [1:28:17<00:00, 185.48it/s] 


In [42]:
X

[array([-0.03395825, -0.67349976,  0.58216596, -0.09959672, -0.99183863,
        -0.5565805 ,  0.11324669, -0.350478  , -0.36015078,  0.40265054,
         0.2140752 ,  1.1076304 ,  0.3734044 ,  0.21968028,  0.62487096,
         1.0441802 , -0.6189276 , -0.11607574,  0.01518528,  0.6328776 ,
        -0.09559837, -1.0673356 , -0.03363014,  0.3349501 ,  0.03966294,
         0.29013422,  1.3662404 ,  0.6177793 ,  0.51331705, -1.10129   ,
        -0.24298452,  0.01146655,  0.13024688,  0.08330432, -0.18535121,
         0.57974225, -0.15472637,  0.66364956, -0.3385564 ,  0.1831999 ,
        -0.7018438 ,  0.3521675 , -0.7843148 , -0.5243667 , -1.0034667 ,
         0.74103826,  0.02359224, -0.19830194, -0.12905021,  0.60237426,
        -0.8283753 ,  0.65788716,  0.08628584,  0.1433344 ,  0.23814979,
        -0.02954806,  0.22417288,  0.03730551, -0.307448  , -0.10726725,
         0.55976105, -0.25997898,  0.13731813, -0.30003116, -0.19776812,
         0.34090528,  0.08723677, -0.19277397, -0.0

In [28]:
import numpy as np
np.save("X_avg_dimensions.npy", X)

In [30]:
x_loaded = np.load("X_avg_dimensions.npy", allow_pickle=True)

In [44]:
x_loaded

array([[-0.03395825, -0.67349976,  0.58216596, ...,  0.48375449,
        -1.02386034,  0.9823693 ],
       [ 0.58818203, -0.46096581,  0.41245058, ...,  0.28636009,
        -0.25543889,  0.22922847],
       [-0.32425007, -0.29042414,  0.54835296, ...,  0.97502953,
        -0.41926903,  0.89624918],
       ...,
       [-0.18181738, -0.79057986,  0.71410042, ..., -0.26437938,
        -1.51680219,  0.51813149],
       [-0.5395022 , -0.99497145,  1.64589906, ...,  1.5755657 ,
        -0.82998741,  0.42581755],
       [ 0.09540737, -0.68519139,  0.93021792, ...,  0.54462487,
        -1.45533538,  0.12679331]])

In [ ]:
###### checking whether any missing data while applying word2vec
print(x_loaded.shape,messages.shape) 

(982619, 100) (982619, 2)


In [ ]:
##checking dependent y shape
messages['overall'].shape

(982619,)

In [ ]:
##converting 100 dimension in one row of each sentence
df=[]
for i in range(0,len(X)):
    df.append(pd.DataFrame(X[i].reshape(1,-1)))

In [68]:
df = np.vstack(df)          # shape becomes (982619, 100)

In [70]:
df = pd.DataFrame(df)

In [73]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.033958,-0.673500,0.582166,-0.099597,-0.991839,-0.556580,0.113247,-0.350478,-0.360151,0.402651,...,-0.346922,0.403974,0.616134,0.282910,0.242640,0.394504,-0.451450,0.483754,-1.023860,0.982369
1,0.588182,-0.460966,0.412451,-0.161210,-1.297049,0.102599,0.485956,-0.624519,-0.644282,0.066047,...,0.017034,0.392730,0.912194,-0.457047,1.074756,0.379170,0.025209,0.286360,-0.255439,0.229228
2,-0.324250,-0.290424,0.548353,-0.255789,-0.826016,0.041851,0.269798,-0.784527,-0.677724,-0.023975,...,-0.764754,0.539960,0.297789,-0.547416,-0.143080,0.574657,-0.148304,0.975030,-0.419269,0.896249
3,-0.430053,-0.520307,1.139486,-0.052925,-0.144256,-0.100470,0.811972,-1.171256,0.400370,-0.187186,...,-0.834716,0.987583,0.900402,0.521543,-0.315098,0.094073,-0.859925,0.871110,-1.029137,0.311490
4,-0.227453,-0.660120,0.635551,-0.432679,-0.971969,-0.421566,0.786697,-0.458815,-0.409131,0.662463,...,-0.278127,0.749875,0.792347,0.232169,-0.531783,0.620726,-0.393760,1.765304,-0.645167,0.296316


In [74]:
##independent features 
x=df
##dependent feature 
y=messages['overall']

In [75]:
#train test split 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30)


In [77]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
784406,-1.465774,-0.461832,1.518442,-0.350879,0.178341,-0.170318,0.765416,-0.008389,0.253160,-0.428105,...,0.295112,-0.419844,0.444940,0.298597,0.645294,-0.439866,-0.960081,0.079859,-0.404130,1.023975
935074,-0.452803,-0.187089,0.306488,-1.844234,-0.661385,0.537020,0.095161,-0.058832,-1.033861,-0.224504,...,1.159427,0.184680,0.466644,-0.301235,1.815604,-0.172591,-0.584555,-0.231902,0.210679,0.403934
449748,-0.389799,-0.298983,0.620025,-0.427202,-0.548296,-0.601524,0.469680,-0.529659,-0.414266,0.257759,...,-0.450447,-0.645564,-0.177094,0.109098,0.658949,0.505235,-0.545179,-0.908304,-0.733845,0.058342
321844,-0.382610,-0.614093,0.553296,-0.078344,-0.969523,-0.149266,0.248836,-0.442928,-0.181275,-0.045716,...,-0.198544,0.386055,0.426902,0.063605,0.735661,0.570119,-0.701203,0.631899,-0.738649,0.521114
420402,-1.049715,-0.590687,0.422421,0.721702,-1.136298,0.206609,0.977829,-0.773224,-0.545093,-0.024976,...,0.090402,1.455299,0.688385,-0.149627,0.479497,0.419128,-0.338990,1.202741,-0.047954,0.107229


In [78]:
y_train

784406    1
935074    1
449748    1
321844    1
420402    1
         ..
821548    1
767594    1
25188     1
921570    1
209623    1
Name: overall, Length: 687833, dtype: int64

In [79]:
###### now applying logistic regression and random forest##########

In [84]:
from sklearn.ensemble import RandomForestClassifier
rfc_model=RandomForestClassifier()
tqdm(rfc_model.fit(x_train,y_train))

  0%|          | 0/100 [00:00<?, ?it/s]

In [85]:
y_pred_word2vec_train=rfc_model.predict(x_train)
y_pred_word2vec_test=rfc_model.predict(x_test)
#performance
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print('-----------------word2vec---------------Random forest-------------------------')
print(f'Training accuracy: {accuracy_score(y_train,y_pred_word2vec_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_word2vec_test)}')
print(f'Confusion matrix: {confusion_matrix(y_test,y_pred_word2vec_test)}')
print(f' classification report: {classification_report(y_test,y_pred_word2vec_test)}')

-----------------word2vec---------------Random forest-------------------------
Training accuracy: 0.9999767385397328
Testing accuracy:0.9480131349521348
Confusion matrix: [[  2443  14766]
 [   559 277018]]
 classification report:               precision    recall  f1-score   support

           0       0.81      0.14      0.24     17209
           1       0.95      1.00      0.97    277577

    accuracy                           0.95    294786
   macro avg       0.88      0.57      0.61    294786
weighted avg       0.94      0.95      0.93    294786



In [89]:
#saving in pkl file 
import pickle
with open("rfc_model_word2vec.pkl", "wb") as f:
    pickle.dump(rfc_model, f)

In [90]:
##applying logistic reg #########
from sklearn.linear_model import LogisticRegression
lgr_model_word2vec=LogisticRegression()
lgr_model_word2vec.fit(x_train,y_train)
y_pred_word2vec_train=lgr_model_word2vec.predict(x_train)
y_pred_word2vec_test=lgr_model_word2vec.predict(x_test)
#performance
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print('-----------------word2vec---------------logistic regression -------------------------')
print(f'Training accuracy: {accuracy_score(y_train,y_pred_word2vec_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_word2vec_test)}')
print(f'Confusion matrix: {confusion_matrix(y_test,y_pred_word2vec_test)}')
print(f' classification report: {classification_report(y_test,y_pred_word2vec_test)}')

-----------------word2vec---------------logistic regression -------------------------
Training accuracy: 0.9503120670278977
Testing accuracy:0.9502995393268338
Confusion matrix: [[  5416  11793]
 [  2858 274719]]
 classification report:               precision    recall  f1-score   support

           0       0.65      0.31      0.43     17209
           1       0.96      0.99      0.97    277577

    accuracy                           0.95    294786
   macro avg       0.81      0.65      0.70    294786
weighted avg       0.94      0.95      0.94    294786



In [91]:
#saving in pkl file 
import pickle
with open("lgr_model_word2vec.pkl", "wb") as f:
    pickle.dump(lgr_model_word2vec, f)